In [ ]:
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import json
from pydantic import BaseModel
from pydantic import TypeAdapter
from langchain_core.runnables import RunnableLambda

In [ ]:
api_key = ""

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    api_key=api_key,
    temperature=0,
)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente técnico e objetivo."),
    ("human", "Explique o que é LangChain em 3 tópicos."),
])

parser = StrOutputParser()

chain = prompt | llm | parser

response = chain.invoke({})
print(response)

In [ ]:
df = pd.read_excel("assets/base_itens.xlsx")
df_raw = df.copy()
df = df[['id_item', 'title_item', 'description_item']]
df['texto'] = df['title_item'] + '-' + df['description_item']
df.drop(columns=['title_item','description_item'], inplace=True)
df.head()

In [ ]:
SYSTEM = """
Você é um validador de políticas do iAutos.

Políticas:
- 3.1 Permitidos: automóveis (sedan, hatch, SUV, picape, minivan), van, furgão, micro-ônibus/ônibus, moto, triciclo, quadriciclo; utilitário, caminhão, camioneta.
- 3.2 PROIBIDOS: ambulância, funerário, guindaste, veículos de competição (kart, corrida, motocross), tratores, bicicletas, reboques, semirreboques, charretes, carroças, bondes, ciclomotores (≤50cc ou equivalente elétrico).
- 3.3 Publicação: NÃO pode incluir telefone/WhatsApp, e-mail, endereços, redes sociais.

Regras:
- Use apenas o texto. Se ambíguo → Não suspeito.
- Se citar categoria proibida (3.2) ou dados de contato (3.3) → Suspeito.

Saída esperada: JSON válido, lista onde cada item tem:
{{
  "id_item": "...",
  "compflg_suspeito_genailiant": true|false,
  "comentario_genai": "descrição1;descrição2;...",
  "reasoning": "..."
}}
"""

PROMPT_ALL = ChatPromptTemplate.from_messages([
    ("system", SYSTEM),
    ("user", "Classifique os seguintes anúncios (lista JSON):\n\n{items}\n\nResponda apenas com JSON válido (lista).")
])

class ItemOut(BaseModel):
  id_item: str
  compflg_suspeito_genailiant: bool
  comentario_genai: str
  reasoning: str

def df_to_records(df):
  return df.astype(str).to_dict(orient="records")

def build_messages(items):
  return PROMPT_ALL.format_messages(items=json.dumps(items, ensure_ascii=False))

def validar_saida(resp_content):
    adapter = TypeAdapter(list[ItemOut])
    models = adapter.validate_json(resp_content)   
    return pd.DataFrame(adapter.dump_python(models))

def invoke(messages):
  resp = llm.invoke(messages)
  data = validar_saida(resp.content) 
  return data

def submit(df):
  records = df_to_records(df)
  msgs = build_messages(records)
  return invoke(msgs)

result = submit(df)

In [ ]:
def make_items_injector(df):
    records = df_to_records(df)
    items_json = json.dumps(records, ensure_ascii=False)
    return RunnableLambda(lambda _: {"items": items_json})

parser = RunnableLambda(lambda text: validar_saida(text))

def build_chain_for_df(df):
    return (
        make_items_injector(df) 
        | PROMPT_ALL
        | llm
        | (lambda msg: msg.content)
        | parser
    )

chain = build_chain_for_df(df)
response = chain.invoke({})  